# Data Augmentations Visualized

This notebook provides visualizations of the data augmentations used in the DeepFont-Torch project. Each augmentation is applied to a sample image, and the results are displayed for comparison.

The main augmentation we focus on is the greyscale gradient overlay, which simulates lighting effects on font images. This augmentation is not well described in the original DeepFont paper, so we provide a detailed implementation and visualization here. We also demonstrate the real and synthetic image augmentation pipelines.

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from deepfont.data.augmentations import (
    augmentation_pipeline,
    add_greyscale_gradient,
)

# Create assets directory
assets_dir = Path("../assets")
assets_dir.mkdir(exist_ok=True)

## Helper Functions and Loading Sample Images

In [ ]:
# Helper functions for loading and displaying images
def load_image(path):
    """Load an image and convert to numpy array."""
    img = Image.open(path).convert("L")  # Convert to grayscale
    return np.array(img, dtype=np.uint8)


def display_images(images, titles, figsize=(15, 5), cmap="gray"):
    """Display multiple images in a row."""
    n = len(images)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axes = [axes]

    for ax, img, title in zip(axes, images, titles):
        ax.imshow(img, cmap=cmap, vmin=0, vmax=255)
        ax.set_title(title, fontsize=12)
        ax.axis("off")

    plt.tight_layout()


def display_grid(images, titles, cols=3, figsize=(15, 10), cmap="gray"):
    """Display multiple images in a grid."""
    n = len(images)
    rows = (n + cols - 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = axes.flatten() if n > 1 else [axes]

    for i, (img, title) in enumerate(zip(images, titles)):
        axes[i].imshow(img, cmap=cmap, vmin=0, vmax=255)
        axes[i].set_title(title, fontsize=10, wrap=True)
        axes[i].axis("off")

    # Hide unused subplots
    for i in range(n, len(axes)):
        axes[i].axis("off")

    plt.tight_layout()


# Load example images
data_dir = Path.cwd().parent / "data"
synthetic_img = load_image(data_dir / "pretraining_examples" / "example_0_label_0.png")
real_img = load_image(data_dir / "pretraining_examples" / "example_0_real.jpeg")

print(f"Loaded synthetic image: {synthetic_img.shape}")
print(f"Loaded real image: {real_img.shape}")

## Greyscale Gradient Overlay

The gradient overlay simulates lighting effects by adding a linear gradient across the image. This helps the model become robust to varying lighting conditions.

In [ ]:
# Apply gradient multiple times to show variation
images = [synthetic_img]
titles = ["Original"]

for i in range(4):
    augmented = add_greyscale_gradient(synthetic_img.copy())
    images.append(augmented)
    titles.append(f"Gradient {i + 1}")

display_images(images, titles, figsize=(18, 4))

save_path = assets_dir / "gradient_examples.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()

## Real and Synthetic Image Augmentation Pipelines

The real image augmentation pipeline includes a series of transformations that are applied to real font images during training. The synthetic image augmentation pipeline includes transformations that are applied to synthetically generated font images. We will visualize the effects of these pipelines on sample images.

In [ ]:
# Apply augmentation pipeline to real image multiple times to show variation
augmented_images = []
augmented_titles = []
for i in range(6):
    augmented = augmentation_pipeline(real_img.copy(), image_type="real", aug_prob=0.5)
    augmented_images.append(augmented)
    augmented_titles.append(f"Augmentation {i + 1}")
print("Real Images with Augmentation Pipeline:")
display_grid(augmented_images, augmented_titles, cols=3, figsize=(8, 4))
save_path = assets_dir / "real_augmentation_examples.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()

# Apply augmentation pipeline to synthetic image multiple times to show variation
augmented_images = []
augmented_titles = []
for i in range(6):
    augmented = augmentation_pipeline(synthetic_img.copy(), image_type="synthetic", aug_prob=0.5)
    augmented_images.append(augmented)
    augmented_titles.append(f"Augmentation {i + 1}")
print("Synthetic Images with Augmentation Pipeline:")
display_grid(augmented_images, augmented_titles, cols=3, figsize=(8, 4))
save_path = assets_dir / "synthetic_augmentation_examples.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()